[![](images/colab-badge.png)](https://colab.research.google.com/github/fzampirolli/si-md2/blob/main/si-md2/notebooks_alunos/cap02/cap02_aluno.ipynb)

# Mineração de Dados e Regras de Associação

Exemplo de figura @fig-2-1.

![Relação entre Dados, Informação e Conhecimento](images/fig1_1.png){#fig-2-1}

Exemplo de tabela-imagem @tbl-2-1.

![Relação entre Dados, Informação e Conhecimento](images/fig1_1.png){#tbl-2-1}

A tabela @tbl-1-X.

| x | y |
|---|---|
| 1 | 2 |

: Legenda da tabela {#tbl-1-X}

A equação @eq-1-1.

$$
x^3
$$ {#eq-1-1}

## Referências do Capítulo


Tem que citar neste capítulo: @han2008, @tan2009, @witten2005, @goldschmidt2005


\printbibliography[heading=none]